# Set working directory

In [2]:
import os
cwd = os.path.split(os.getcwd())
if cwd[-1] == 'tutorials':
    os.chdir('..')
    print(f'Changed directory to {os.getcwd()}')
!python --version

Python 3.7.4


# Import modules

In [3]:
from BRON_db.build_BRON_db import build_graph
from path_search.path_search_BRON_db import main_attack
from utils.bron_network_utils import load_graph_nodes

# Build BRON

The different types of threat data can be linked together in a graph-based format called BRON_db. To create a new BRON_db using input threat data, run the following command:
```
python BRON_db/build_BRON_db.py --data_folder DATA_FOLDER --save_path FILE_NAME --only_recent_cves (optional)
``` 
`DATA_FOLDER` is the folder path of input data, and `FILE_NAME` is the file path to save BRON_db. To make BRON_db using only recent CVEs, add the argument `--only_recent_cves`. An example BRON_db using input threat data in the `example_data` folder can be found in `data/BRON_db.json`.

In [4]:
save_path = 'data/BRON_db.json'
input_data_folder = 'example_data'
build_graph(save_path, input_data_folder)

# Network specific BRON

In addition to creating graph-like databases of the threat data, BRON can create graphs that connect to specific nodes of a given network configuration. These databases are called "network_specific_BRON_db". Note that "network_specific_BRON_db" contain all the node information as a non-network-specific BRON_db (described above). A comparison of the data links between the two types of BRON_db can be seen below:

Data links shown in non-network-specific graph (BRON_db):
```
TACTIC --> TECHNIQUE --> CAPEC --> CWE --> CVE --> CPE
```
Data links shown in network-specific graphs (network_specific_BRON_db):
```
TACTIC --> TECHNIQUE --> CAPEC --> CWE --> CVE --> CPE --> Network Node
```

# Structure of BRON_db

Each entry of threat data is a node in BRON_db and has 4 attributes. The node has a unique name in BRON\_db that is of the form (threat data type)\_(unique 5 digit id) where the threat data type is either tactic, technique, capec, cwe, cve, cpe, or network-node.

Additionally, there are four attributes for each node:
- Original_id: ID of threat data in MITRE/NIST if it exists
- Datatype: One of tactic, technique, capec, cwe, cve, cpe, or network-node
- Name: Name of threat data in MITRE/NIST if exists
- Metadata: Any additional information that is contained in MITRE/NIST 

# Find paths in BRON_db

Once the BRON_db is built, it can be searched for paths that contain certain data types or specific nodes. The path search starts with an initial set of starting points and then finds all the paths in BRON_db from those starting points. A severity score based on the CVSS score is calculated for each path that is found. To find paths on a non-network-specific BRON_db, run the following command:
```
python -m path_search.path_search_BRON_db --db_path DB_PATH --starting_point STARTING_POINT --starting_point_type STARTING_POINT_TYPE --results_file RESULTS_FILE --length (optional) --network_specific (optional)
```
`DB_PATH` is the file path of the BRON_db that is being searched, `STARTING_POINT` is the file path of the CSV file that contains the IDs of the starting point(s), `STARTING_POINT_TYPE` is the data type of the starting points (either tactic, technique, capec, cwe, cve, or cpe), and `RESULTS_FILE` is the file path to store the CSV results file.

To make the results contain only the _number_ of nodes in a path (instead of a set of node names), add the argument `--length`. If the BRON_db that is being searched is network_specific, then add the argument `--network_specific`.

The output of path search is a CSV file that shows how the different data types are related to each other. The paths start at the type of argument and then follow the graph to find the connections to other data. For example, if my input file consisted of three CAPECs: 100, 105, 200, then we would see all of the Tactics, Techniques, CWEs, CVEs, and CPEs associated with each CAPEC individually. The final row contains the total count of each data type in BRON_db.

An example starting point file and its path search result can be found in `example_data`.

In [5]:
db_path = save_path # use same BRON_db created above
starting_point = 'example_data/starting_point_tactic.csv'
starting_point_type = 'tactic'
results_file = 'data/search_result_tactic.csv'
length = False
network_specific = False
main_attack(db_path, starting_point, results_file, starting_point_type, length, network_specific)

,tactic,technique,capec,cwe,cve,risk_score,cpe,attack,Node Type
0,{tactic_00008},"{technique_00216, technique_00188, technique_0...","{capec_00286, capec_00302, capec_00301, capec_...","{cwe_00520, cwe_00409, cwe_00521, cwe_00987, c...","{cve_01167, cve_01459, cve_01045, cve_01536}",25.70,"{cpe_01044, cpe_01168, cpe_01533, cpe_01458}",NaN,connected
1,{tactic_00014},"{technique_00027, technique_00212, technique_0...","{capec_00295, capec_00336, capec_00293, capec_...","{cwe_00914, cwe_00412, cwe_00502, cwe_00346}","{cve_01144, cve_01464, cve_01475, cve_01498, c...",101.50,"{cpe_01137, cpe_01120, cpe_01250, cpe_01469, c...",NaN,connected
2,{tactic_00012},"{technique_00216, technique_00036, technique_0...","{capec_00303, capec_00291, capec_00301, capec_...","{cwe_00359, cwe_00651, cwe_00373, cwe_00449, c...","{cve_01143, cve_01081, cve_01254, cve_01457, c...",101.95,"{cpe_01431, cpe_01289, cpe_01299, cpe_01412, c...",NaN,connected
3,12,266,519,251,217,1014.00,277,0.0,Total Number
